# Lesson 1 // Data wrangling 

---
In this lesson we'll:

1. see how to save .csv data in the form of an .RData object
2. introduce data transformations using the **dplyr** package, using the five key dplyr verbs: 

    + filter
    + arrange
    + mutate
    + summarise
    + grouped_by


3. introduce the pipe operator `%>%`
4. see a few nice things you can do by combining dplyr verbs (grouped filters and mutates, for example)
5. use the dplyr verbs to build a small movie rating dataset that we'll use in the next lesson on recommender systems.
6. introduce various *join* operations that can be used to combine information across multiple tables (relational data)

### Sources and references

 * http://r4ds.had.co.nz/transform.html
 * http://r4ds.had.co.nz/relational-data.html
 ---

## Get the MovieLens data and save as .RData

[MovieLens](https://grouplens.org/datasets/movielens/) is a great resource for data on movie ratings. The full dataset has ratings on 40 000 movies by 260 000 users, some 24 million ratings in all. We'll use a smaller dataset with ratings of 9 000 movies by 700 users (100 000 ratings in all). 

Download the file "ml-latest-small.zip" from https://grouplens.org/datasets/movielens/ and unzip it to the *data* directory in your main project folder (make a folder called *data* if you haven't already). You should see four csv files: `links.csv`, `movies.csv`, `ratings.csv`, and `tags.csv`.


First let's save the data we downloaded as an .RData object. .RData objects are smaller than csv, plus we can save all four csvs in a single .RData object that we can call with a single call to `load` the dataset later on.

In [ ]:
# read in the csv files
links <- read.csv("data/ml-latest-small/links.csv")
movies <- read.csv("data/ml-latest-small/movies.csv")
ratings <- read.csv("data/ml-latest-small/ratings.csv")
tags <- read.csv("data/ml-latest-small/tags.csv")

# save as .RData
save(links,movies,ratings,tags,file="data/movielens-small.RData")

# check that its worked
rm(list=ls())
load("data/movielens-small.RData")

You'll only need to do the above part once so, once you've got the data saved as .RData, start running the notebook from here.

## Loading the tidyverse

Load the **tidyverse** collection of packages, which loads the following packages: **ggplot2**, **tibble**, **tidyr**, **readr**, **purrr**, and **dplyr**.

In [ ]:
library(tidyverse)

Load the MovieLens data

In [ ]:
load("data/movielens-small.RData")

Tibbles are a special kind of dataframe that work well with tidyverse packages ("in the tidyverse" in tidyversese). 

In [ ]:
# convert ratings to a "tibble"
ratings <- as.tibble(ratings)

A nice feature of tibbles is that if you display them in the console (by typing `ratings`, for example) only first few rows and columns are shown. Unfortunately this doesn't carry over to jupyter notebook, so I need to explicitly say `print(ratings)` or all the rows are shown.

In [ ]:
print(ratings)

Explore some of the variables

In [ ]:
str(ratings)

In [ ]:
glimpse(ratings)

In [ ]:
glimpse(movies)

We'll look at database joins in more detail, but for now, this just adds movie title to the `ratings` data by pulling that information from `movies`.

In [ ]:
ratings <- left_join(ratings, movies)

## Filtering rows with `filter()`

Here we illustrate the use of `filter()` by extracting user 1's observations from the *ratings* data frame.

In [ ]:
u1 <- filter(ratings, userId == 1)
u1

Next we extract the observations for user 1 that received a rating greater than 3. Multiple filter conditions are created with `&` (and) and `|` (or).

In [ ]:
filter(ratings, userId == 1 & rating > 3)

Here's another way of writing the same condition as above:

In [ ]:
filter(ratings, userId == 1, rating > 3)

The `%in%` command is often useful when using dplyr verbs:

In [ ]:
filter(ratings, userId == 1, rating %in% c(1,4))

## Introducing the pipe

The pipe operator `%>%` is a very useful way of chaining together multiple operations. A typical format is something like:

*data* `%>%` *operation 1* `%>%` *operation 2* 

You read the code from left to right: Start with *data*, apply some operation (operation 1) to it, get a result, and then apply another operation (operation 2) to that result, to generate another result (the final result, in this example). A useful way to think of the pipe is as similar to "then".

The main goal of the pipe is to make code easier, by focusing on the transformations rather than on what is being transformed. Usually this is the case, but its also possible to get carried away and end up with a huge whack of piped statements. Deciding when to break a block up is an art best learned by experience. 

In [ ]:
# filtering with the pipe
ratings %>% filter(userId == 1)

The main usefulness of the pipe is when combining multiple operations

In [ ]:
# first filter on userId then on rating
u1_likes <- ratings %>% filter(userId == 1) %>% filter(rating > 3)
u1_likes

In [ ]:
# another way of doing the same thing
ratings %>% filter(userId == 1 & rating > 3)

## Arranging rows with `arrange()`

Ordering user 1's "liked" movies in descending order of rating (note the use of `desc`)

In [ ]:
arrange(u1_likes, desc(rating))

Subsequent arguments to `arrange()` can be used to arrange by multiple columns. Here we first order user 1's liked movies by rating (in descending order) and then by timestamp (in ascending order)

In [ ]:
arrange(u1_likes, desc(rating),timestamp)

We can also use the pipe to do the same thing

In [ ]:
u1_likes %>% arrange(desc(rating))

Finally, here's an example of combining filter and arrange operations with the pipe

In [ ]:
ratings %>% filter(userId == 1 & rating > 3) %>% arrange(desc(rating))

## Selecting columns with `select()`

Select is a bit like `filter()` for columns. The syntax is straightforward, the first argument gives the dataframe, and then you list the variables you want to select!

In [ ]:
select(u1_likes,title,rating)

To exclude variables just put a minus sign in front of them

In [ ]:
select(u1_likes,-userId,-timestamp)

You can also use `select()` to reorder variables. A useful helpful function here is `everything()`.

In [ ]:
# original order
u1_likes

# reorder so title is first
select(u1_likes, title, everything())

## Adding new variables with `mutate()`

Mutating operations add a new column to a dataframe. Here's a trivial example to get started:

In [ ]:
mutate(u1_likes, this_is = "stupid")  

A more useful use of mutate is to construct new variable based on existing variables. This is the way that `mutate` is almost always used.

In [ ]:
mutate(u1, like = ifelse(rating > 3, 1, 0))  

We can also use the pipe for mutating operations. Hopefully you're getting used to the pipe by now, so let's embed a mutating operation within a larger pipe than we've used before. 

In [ ]:
ratings %>% 
mutate(like = ifelse(rating > 3, 1, 0)) %>% 
filter(userId == 1) %>% 
select(like, everything()) 

## Aggregating over rows with `summarise()`

The `summarise()` verb (or `summarize()` will also work) summarises the rows in a data frame in some way. When applied to the whole data frame, it will collapse it to a single row. For example, here we take user 1's data, and calculate their average rating and the number of movies they have given a rating higher than 3 to:

In [ ]:
summarise(u1, mean = mean(rating), likes = sum(rating > 3))

You need to watch out for NAs when using `summarise()`. If one exists, operations like `mean()` will return NA. You can exclude NAs from calculations using `na.rm = TRUE`:

In [ ]:
# introduce an NA
u1$rating[1] <- NA

# see what happens
summarise(u1, mean = mean(rating), likes = sum(rating > 3))

In [ ]:
# with na.rm = T
summarise(u1, mean = mean(rating, na.rm = T), likes = sum(rating > 3, na.rm = T))

`summarise()` is most useful when combined with `group_by()`, which imposes a grouping structure on a data frame. After applying `group_by()`, subsequent dplyr verbs will be applied to individual groups, basically repeating the code for each group. That means that `summarise()` will calculate a summary for each group:

In [ ]:
# tell dplyr to group ratings by userId
ratings_by_user <- group_by(ratings, userId)

# apply summarize() to see how many movies each user has rated
ratings_by_user %>% summarize(count = n()) %>% head()

In [ ]:
# get sorted counts (plus some presentation stuff)
ratings %>% 
group_by(userId) %>% 
summarize(count = n()) %>% 
arrange(desc(count)) %>% 
head(20) %>%     # take first two rows
t()  # transpose 

In [ ]:
# or with the pipe (last time)
ratings %>% group_by(userId) %>% summarize(count = n()) %>% head(10)

## Other uses of `grouped_by()`: grouped filters and grouped mutates
While you'll probably use `group_by()` most often with `summarise()`, it can also be useful when used in conjunction with `filter()` and `mutate()`. Grouped filters perform the filtering within each group. Below we use it to extract each user's favourite movie (or movies, if there's a tie).

In [ ]:
# example of a grouped filter
ratings %>% group_by(userId) %>% filter(rank(desc(rating)) < 2)

Here we use a grouped mutate to standardise each user's ratings so that they have a mean of zero (for each user, which guarantees the overall mean rating is also zero).

In [ ]:
# example of a grouped mutate
ratings %>% 
group_by(userId) %>%
mutate(centered_rating = rating - mean(rating)) %>% 
select(-movieId,-timestamp,-genres)

## Putting it all together: extracting a sample set of reviews for Lesson 2

In this section we'll take what we've learned and do something useful: build a 15x20 matrix containing the reviews made on 20 movies by 15 users. We'll use this matrix in the next lesson to build a recommendation system.

First, we select the 15 users we want to use. I've chosen to use 15 users with moderately frequent viewing habits (remember there are 700 users and 9000 movies), mainly to make sure there are some (but not too many) empty ratings.

In [ ]:
users_frq <- ratings %>% group_by(userId) %>% summarize(count = n()) %>% arrange(desc(count))
my_users <- users_frq$userId[101:115]

Next, we select the 20 movies we want to use:

In [ ]:
movies_frq <- ratings %>% group_by(movieId) %>% summarize(count = n()) %>% arrange(desc(count))
my_movies <- movies_frq$movieId[101:120]

Now we make a dataset with only those 15 users and 20 movies:

In [ ]:
ratings_red <- ratings %>% filter(userId %in% my_users, movieId %in% my_movies) 
# and check there are 15 users and 20 movies in the reduced dataset
n_users <- length(unique(ratings_red$userId))
n_movies <- length(unique(ratings_red$movieId))
print(paste("number of users is",n_users))
print(paste("number of movies is",n_movies))

Let's see what the 20 movies are:

In [ ]:
movies %>% filter(movieId %in% my_movies) %>% select(title)

However, note all the movie titles are still being kept:

In [ ]:
levels(ratings_red$title)

This actually isn't what we want, so let's drop the ones we won't use.

In [ ]:
ratings_red <- droplevels(ratings_red)
levels(ratings_red$title)

We now want to reshape the data frame into a 15x20 matrix i.e.from "long" format to "wide" format. We can do this using the `spread()` verb. 

In [ ]:
ratings_red %>% spread(key = movieId, value = rating)

The preceding line *doesn't* work: as you can see we land up with more than one row per user. But it is useful as an illustration of `spread()`. Question: why doesn't it work?

Here's the corrected version:

In [ ]:
ratings_red %>% select(userId,title,rating) %>% spread(key = title, value = rating)

Finally, since we just want to know who has seen what, we replace all NAs with 0 and all other ratings with 1:

In [ ]:
viewed_movies <- ratings_red %>% 
  select(userId,title,rating) %>% 
  complete(userId, title) %>% 
  mutate(seen = ifelse(is.na(rating),0,1)) %>% 
  select(userId,title,seen) %>% 
  spread(key = title, value = seen)

We could have got this more simply with a call to `table()`, which creates a two-way frequency table.

In [ ]:
table(ratings_red$userId,ratings_red$title)

Finally, we save our output for use in the next lesson!

In [ ]:
save(ratings_red, viewed_movies, file = "output/recommender.RData")

## Combining data frames with *joins*

We'll often need to combine the information contained in two or more tables. To do this, we need various kinds of database *joins*. This section describes the basic join operations that we need to combine data frames. The examples are taken from [Chapter 13](http://r4ds.had.co.nz/relational-data.html) of R4DS, which also contains a lot more general information on relational data.

First, we make some very simple data tables to show how the joins work:

In [ ]:
# make some example data
x <- tribble(
  ~key, ~xvalue,
  1, "x1",
  2, "x2",
  3, "x3"
)

y <- tribble(
  ~key, ~yvalue,
  1, "y1",
  2, "y2",
  4, "y3"
)

### Mutating joins: `inner_join`, `left_join`, `right_join`, `full_join`

The first set of joins we look at are called mutating joins. These first match observations in two tables in some way, and then combine variables from the two tables.

There are four types of mutating joins: inner joins, left joins, right joins, and full joins. 

An **inner join** keeps observations that appear in *both* tables. 

In [ ]:
inner_join(x,y)

In [ ]:
inner_join(y,x)

The other three joints are all **outer joins**: they keep observations that appear in *at least one* of the tables.

A **left join** keeps all observations in x.

In [ ]:
left_join(x,y)

In [ ]:
left_join(y,x)

A **right join** keeps all observations in y.

In [ ]:
# note this is the same as left_join(y,x)
right_join(x,y)

A **full join** keeps observations in x or y.

In [ ]:
full_join(x,y)

We can now re-examine the join we used to add movie titles to the ratings data frame earlier:

In [ ]:
# reload the MovieLens data
load("data/movielens-small.RData")
ratings <- as.tibble(ratings)
movies <- as.tibble(movies)

Note that the same *movieId* can appear multiple times in the *ratings* data frame:

In [ ]:
print(ratings %>% arrange(movieId)) # note duplicate movieIds

But each *movieId* only appears once in the *movies* data frame:

In [ ]:
print(movies %>% arrange(movieId)) # note unique movieIds

In this case a left join by the *movieId* key copies across the movie title information (as well as any other information in the *movies* data frame):

In [ ]:
print(left_join(ratings, movies, by = "movieId") %>% select(title,everything()))

### Filtering joins: `semi_join`, `anti_join`

The last two joins we look at are **filtering joins**. These match observations in two tables, but do not add variables. There are two types of filtering joins: semi-joins and anti-joins.

A **semi join** keeps all observations in x that appear in y (note variables are from x only),

In [ ]:
semi_join(x,y)

In [ ]:
semi_join(y,x)

while an **anti join** *drops* all observations in x that appear in y (note variables are from x only).

In [ ]:
anti_join(x,y)

In [ ]:
anti_join(y,x)

## Exercises

Do the exercises in [Chapter 5](http://r4ds.had.co.nz/transform.html) (data transformation using the **dplyr** verbs) and [Chapter 13](http://r4ds.had.co.nz/relational-data.html) (on database joins) of R4DS. There are exercises at the end of each major subsection. Do as many of these exercises as you need to feel comfortable with the material - I suggest doing at least the first two of each set of exercises.